# Netoyage des données

## Les  imports

In [23]:
# !pip install fastparquet

In [154]:
# !pip install -U googlemaps

In [157]:
# !pip install geopy

In [118]:
import pandas as pd
import os

In [119]:
## Modifier le WorkDirectory
wdir = '/tmp/gas'
os.chdir(wdir)
print('current work dir is:', os.getcwd())

current work dir is: /tmp/gas


In [120]:
# limiter le nombre de lignes à afficher à 10
# au lieu de faire un head() + un tail() il suffira d'afficher la DataFrame
pd.options.display.max_rows = 10

## Fonction pour detecter les colonnes susebtible de devenir des categories

In [121]:
def findLargeCols(df):
    head = '{:35} {:15} {:14} {:}'.format('col_name',
                             'n',
                             'n_distinct',
                             'ratio')
    print(head)
    for col_name in list(df):
        n          = len(df[col_name])
        n_distinct = len(df[col_name].unique())
        ratio      = n/n_distinct
        row = '{:20} {:20} {:20} {:8.0f}%'.format(col_name,
                                 n,
                                 n_distinct,
                                 ratio)
        print(row)

**Plus le Ratio est grand plus la variable a peu de modalités. c à d, les même valeurs se repetent souvent**

## Typage des DataFrames

### netoyage de **pdv**

In [122]:
pdv = pd.read_csv('pdv.csv', low_memory=False)

### Les valeurs manquantes

In [123]:
pd.DataFrame(pdv.isna().sum()).T

,annee,id,latitude,longitude,cp,pop,ouverture_debut,ouverture_fin,ouverture_saufjour,adresse,ville,rupture_id,rupture_nom,rupture_debut,rupture_fin,fermeture_type,fermeture_debut,fermeture_fin
0,0,0,1006,984,0,0,12598,12598,115538,0,3,0,0,0,92699,43,43,96861


In [124]:
pdv

,annee,id,latitude,longitude,cp,pop,ouverture_debut,ouverture_fin,ouverture_saufjour,adresse,ville,rupture_id,rupture_nom,rupture_debut,rupture_fin,fermeture_type,fermeture_debut,fermeture_fin
0,2019,1000001,4.620114e+06,519791.000000,01000,R,NaN,NaN,NaN,596 AVENUE DE TREVOUX,SAINT-DENIS-LèS-BOURG,4,GPLc,2017-09-16T09:50:23,NaN,NaN,NaN,NaN
1,2019,1000002,4.621842e+06,522767.000000,01000,R,NaN,NaN,NaN,16 Avenue de Marboz,BOURG-EN-BRESSE,5,E10,2017-09-07T11:05:59,NaN,NaN,NaN,NaN
2,2019,1000004,4.618800e+06,524500.000000,01000,R,NaN,NaN,NaN,chal Juin,Bourg-en-Bresse,5,E10,2018-04-21T15:38:51,NaN,NaN,NaN,NaN
3,2019,1000005,4.620094e+06,519942.024022,01000,R,NaN,NaN,NaN,voux,SAINT-DENIS-LèS-BOURG,2,SP95,2011-06-01T00:00:00,NaN,NaN,NaN,NaN
4,2019,1000006,4.620754e+06,523758.000000,01000,R,NaN,NaN,NaN,1 Boulevard John Kennedy,BOURG-EN-BRESSE,3,E85,2017-09-25T19:45:38,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136707,2016,95870005,4.893676e+06,220715.000000,95870,R,06:00:43,21:00:43,NaN,ri,BEZONS,6,SP98,2016-07-20T11:07:00,2017-01-02T11:01:34,D,2016-11-03T00:11:00,2017-01-02T11:01:34
136708,2016,95870007,4.892824e+06,221140.000000,95870,R,06:00:01,21:00:01,NaN,ri,BEZONS,2,SP95,2011-08-31T22:08:00,2017-01-02T11:01:34,D,2016-11-03T00:11:00,2017-01-02T11:01:34
136709,2016,95870009,4.892770e+06,221346.318047,95870,R,06:00:00,21:30:00,NaN,151 AV GABRIEL PERI,BEZONS,2,SP95,2011-08-31T22:08:00,2017-01-02T11:01:34,D,2016-11-03T00:11:00,2017-01-02T11:01:34
136710,2016,96600001,4.875637e+06,241048.000000,94600,R,07:30:27,20:30:27,Dimanche,16 Avenue de Newburn,Choisy-le-Roi,1,Gazole,2009-12-21T20:12:00,2017-01-02T11:01:34,D,2016-11-03T00:11:00,2017-01-02T11:01:34


In [125]:
ad = list(pdv.loc[pdv.ville.isna()].loc[:,'adresse'].unique())[0]
ad

'le bleymard'

In [126]:
pdv.loc[pdv.cp=="35***"] == "35000"

,annee,id,latitude,longitude,cp,pop,ouverture_debut,ouverture_fin,ouverture_saufjour,adresse,ville,rupture_id,rupture_nom,rupture_debut,rupture_fin,fermeture_type,fermeture_debut,fermeture_fin
80571,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [11]:
from geopy.geocoders import Nominatim

ModuleNotFoundError: No module named 'geopy'

In [ ]:
geolocator = Nominatim(user_agent="nominatim.openstreetmap.org")
location = geolocator.reverse("46.201140,  5.197910")
print(location.raw)

In [ ]:
!pip install pgeocode

In [12]:
import pgeocode
nomi = pgeocode.Nominatim('fr')
nomi.query_postal_code("75013")

ModuleNotFoundError: No module named 'pgeocode'

In [127]:
pdv.loc[(pdv.adresse == ad) & (pdv.ville.isna())]

,annee,id,latitude,longitude,cp,pop,ouverture_debut,ouverture_fin,ouverture_saufjour,adresse,ville,rupture_id,rupture_nom,rupture_debut,rupture_fin,fermeture_type,fermeture_debut,fermeture_fin
5865,2019,48190001,4448600.000,373500.000,48190,R,NaN,NaN,NaN,le bleymard,NaN,4,GPLc,2018-03-05T09:41:34,NaN,T,2010-10-26T00:00:00,NaN
29224,2018,48190001,4448600.000,373500.000,48190,R,01:00,01:00,NaN,le bleymard,NaN,5,E10,2018-03-05T09:41:34,NaN,D,2018-09-12T00:00:00,NaN
41563,2017,48190001,3.735,44.486,48190,R,01:00:00,01:00:00,NaN,le bleymard,NaN,2,SP95,2013-06-30T00:00:00,2017-10-13T18:35:08,D,2017-12-05T00:00:00,NaN


In [128]:
pdv.isna().sum()

annee                  0
id                     0
latitude            1006
longitude            984
cp                     0
                   ...  
rupture_debut          0
rupture_fin        92699
fermeture_type        43
fermeture_debut       43
fermeture_fin      96861
Length: 18, dtype: int64

In [15]:
# normalisation des longitudes et latitude 
pdv.loc[:,'latitude']   = pdv.latitude/100000
pdv.loc[:, 'longitude'] = pdv.longitude/100000

In [32]:
pdv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136712 entries, 0 to 136711
Data columns (total 18 columns):
annee                 136712 non-null int64
id                    136712 non-null int64
latitude              135706 non-null float64
longitude             135728 non-null float64
cp                    136712 non-null object
pop                   136712 non-null object
ouverture_debut       124114 non-null object
ouverture_fin         124114 non-null object
ouverture_saufjour    21174 non-null object
adresse               136712 non-null object
ville                 136709 non-null object
rupture_id            136712 non-null int64
rupture_nom           136712 non-null object
rupture_debut         136712 non-null object
rupture_fin           44013 non-null object
fermeture_type        136669 non-null object
fermeture_debut       136669 non-null object
fermeture_fin         39851 non-null object
dtypes: float64(2), int64(3), object(13)
memory usage: 18.8+ MB


- **convertion des categories**

In [17]:
findLargeCols(pdv)

col_name                            n               n_distinct     ratio
annee                              136712                   13    10516%
id                                 136712                16811        8%
latitude                           136712                19704        7%
longitude                          136712                19974        7%
cp                                 136712                 4467       31%
pop                                136712                    4    34178%
ouverture_debut                    136712                  765      179%
ouverture_fin                      136712                  880      155%
ouverture_saufjour                 136712                   31     4410%
adresse                            136712                13901       10%
ville                              136712                 9171       15%
rupture_id                         136712                    6    22785%
rupture_nom                        136712          

les columns **'annee','pop','ouverture_saufjour','rupture_id','rupture_nom','fermeture_type'** mieux vaut les considérer des categories

In [18]:
pdv['annee']              = pdv['annee'].astype('category')
pdv['pop']                = pdv['pop'].astype('category')
pdv['ouverture_saufjour'] = pdv['ouverture_saufjour'].astype('category')
pdv['rupture_id']         = pdv['rupture_id'].astype('category')
pdv['rupture_nom']        = pdv['rupture_nom'].astype('category')
pdv['fermeture_type']     = pdv['fermeture_type'].astype('category')


- **convertion des dates et temps**

In [129]:
mask = pdv['ouverture_debut'].str.len() == 5
pdv.loc[mask,'ouverture_debut'] = pdv.loc[mask,'ouverture_debut'] + ":00"

In [134]:
mask = pdv['ouverture_fin'].str.len() == 5
pdv.loc[mask,'ouverture_fin'] = pdv.loc[mask,'ouverture_fin'] + ":00"

In [76]:
mask = pdv['rupture_debut'].str[:4] == "0216"
pdv.loc[mask,'rupture_debut'] = "2016" + pdv[mask]['rupture_debut'].str[4:]

In [77]:
mask = pdv['rupture_debut'].str[:4] == "0216"
pdv[mask]['rupture_debut']

Series([], Name: rupture_debut, dtype: object)

In [135]:
pdv['ouverture_debut'] = pd.to_timedelta((pdv['ouverture_debut']))
pdv['ouverture_fin']   = pd.to_timedelta((pdv['ouverture_fin']))

In [140]:
pdv['ouverture_debut'][136707] , pdv['ouverture_fin'][136707]

(Timedelta('0 days 06:00:43'), Timedelta('0 days 21:00:43'))

In [143]:
pdv['ouverture_fin'][136707] - pdv['ouverture_debut'][136707]

Timedelta('0 days 15:00:00')

In [136]:
pdv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136712 entries, 0 to 136711
Data columns (total 18 columns):
annee                 136712 non-null int64
id                    136712 non-null int64
latitude              135706 non-null float64
longitude             135728 non-null float64
cp                    136712 non-null object
pop                   136712 non-null object
ouverture_debut       124114 non-null timedelta64[ns]
ouverture_fin         124114 non-null timedelta64[ns]
ouverture_saufjour    21174 non-null object
adresse               136712 non-null object
ville                 136709 non-null object
rupture_id            136712 non-null int64
rupture_nom           136712 non-null object
rupture_debut         136712 non-null object
rupture_fin           44013 non-null object
fermeture_type        136669 non-null object
fermeture_debut       136669 non-null object
fermeture_fin         39851 non-null object
dtypes: float64(2), int64(3), object(11), timedelta64[ns](2)
memo

In [22]:
pdv['rupture_debut']   = pd.to_datetime(pdv['rupture_debut'])
pdv['rupture_fin']     = pd.to_datetime(pdv['rupture_fin'])
pdv['fermeture_debut'] = pd.to_datetime(pdv['fermeture_debut'])
pdv['fermeture_fin']   = pd.to_datetime(pdv['fermeture_fin'])

In [23]:
pdv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136712 entries, 0 to 136711
Data columns (total 18 columns):
annee                 136712 non-null category
id                    136712 non-null int64
latitude              135706 non-null float64
longitude             135728 non-null float64
cp                    136712 non-null object
pop                   136712 non-null category
ouverture_debut       124114 non-null timedelta64[ns]
ouverture_fin         124114 non-null timedelta64[ns]
ouverture_saufjour    136712 non-null category
adresse               136712 non-null object
ville                 136709 non-null object
rupture_id            136712 non-null category
rupture_nom           136712 non-null category
rupture_debut         136712 non-null datetime64[ns]
rupture_fin           44013 non-null datetime64[ns]
fermeture_type        136669 non-null category
fermeture_debut       136669 non-null datetime64[ns]
fermeture_fin         39851 non-null datetime64[ns]
dtypes: category(6

In [98]:
pdv

,annee,id,latitude,longitude,cp,pop,ouverture_debut,ouverture_fin,ouverture_saufjour,adresse,ville,rupture_id,rupture_nom,rupture_debut,rupture_fin,fermeture_type,fermeture_debut,fermeture_fin
0,2007,1000001,4.620114e+06,519791.000000,01000,R,01:00:00,01:00:00,NaN,ROUTE NATIONALE,S-BOURG,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007,1000002,4.621842e+06,522767.000000,01000,R,08:45:00,19:30:00,Dimanche,16 Avenue de Marboz,BOURG-EN-BRESSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007,1000003,4.619965e+06,524047.000000,01000,R,16:00:00,20:00:00,Samedi;Dimanche,Bld Charles De Gaulle,Bourg-en-Bresse,NaN,NaN,NaN,NaN,D,2007-10-30 09:47:04.590533,NaN
3,2007,1000004,4.618836e+06,524447.000000,01000,R,01:00:00,01:00:00,NaN,chal Juin,Bourg-en-Bresse,NaN,NaN,NaN,NaN,D,2007-10-30 09:47:04.590533,NaN
4,2007,1000006,4.620754e+06,523758.000000,01000,R,01:00:00,01:00:00,Dimanche,1 Boulevard John Kennedy,BOURG-EN-BRESSE,NaN,NaN,NaN,NaN,D,2007-10-30 09:47:04.590533,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140719,2017,95870005,4.893676e+06,220715.000000,95870,R,06:00:43,21:00:43,NaN,ri,BEZONS,2.0,SP95,2013-03-20T16:58:00,NaN,D,2017-12-19T00:00:00,NaN
140720,2017,95870007,4.892824e+06,221140.000000,95870,R,06:00:01,21:00:01,NaN,ri,BEZONS,2.0,SP95,2011-08-31T22:43:00,NaN,D,2017-12-19T00:00:00,NaN
140721,2017,95870009,4.892770e+06,221346.318047,95870,R,06:00:00,21:30:00,NaN,151 AV GABRIEL PERI,BEZONS,2.0,SP95,2011-08-31T22:43:00,NaN,D,2017-12-19T00:00:00,NaN
140722,2017,96600001,4.875637e+06,241048.000000,94600,R,07:30:27,20:30:27,Dimanche,16 Avenue de Newburn,Choisy-le-Roi,5.0,E10,2009-12-11T21:53:00,NaN,D,2017-12-19T00:00:00,NaN


- **enregistrer la df compréssée**

In [15]:
pdv.to_parquet('pdv.gzip', 
               compression='gzip')

- **liberer la mémoire**

In [144]:
del pdv

### netoyage de **prix**

In [ ]:
prix = pd.read_csv('prix.csv', low_memory=False)

In [18]:
prix

,annee,id_pdv,nom,id,maj,valeur
0,2017,1000001,Gazole,1,2017-01-02T09:37:03,1258
1,2017,1000001,Gazole,1,2017-01-03T09:54:58,1258
2,2017,1000001,Gazole,1,2017-01-06T12:33:57,1258
3,2017,1000001,Gazole,1,2017-01-09T08:59:53,1258
4,2017,1000001,Gazole,1,2017-01-10T10:38:39,1258
...,...,...,...,...,...,...
28552949,2012,95870008,E10,5,2012-12-24 08:12:41.12514,1446
28552950,2012,95870008,E10,5,2012-12-26 09:59:50.402522,1453
28552951,2012,95870008,E10,5,2012-12-27 06:37:54.702091,1449
28552952,2012,95870008,E10,5,2012-12-28 10:30:37.417588,1448


In [19]:
prix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28552954 entries, 0 to 28552953
Data columns (total 6 columns):
annee     int64
id_pdv    int64
nom       object
id        int64
maj       object
valeur    int64
dtypes: int64(4), object(2)
memory usage: 1.3+ GB


**la DataFrame prix est trés grande (1,3 GB).On peut diminuer sa taille en convertant des variables en Category.**

- **convertion des categories**

In [20]:
findLargeCols(prix)

col_name                            n               n_distinct     ratio
annee                            28552954                   13  2196381%
id_pdv                           28552954                12179     2344%
nom                              28552954                    6  4758826%
id                               28552954                    6  4758826%
maj                              28552954             11299902        3%
valeur                           28552954                 1943    14695%


les columns **'id_pdv','annee','nom','id'** mieux vaut les considérer des categories. on laisse **'valeur'** en **int**

In [21]:
prix["id_pdv"] = prix["id_pdv"].astype('category')
prix["annee"]  = prix["annee"].astype('category')
prix["nom"]    = prix["nom"].astype('category')
prix["id"]     = prix["id"].astype('category')

In [22]:
prix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28552954 entries, 0 to 28552953
Data columns (total 6 columns):
annee     category
id_pdv    category
nom       category
id        category
maj       object
valeur    int64
dtypes: category(4), int64(1), object(1)
memory usage: 572.2+ MB


**la taille de la df diminue de 1.3GB à 0.57GB**

- **convertion des dates et temps**

In [23]:
prix['maj'] = pd.to_datetime(prix['maj'])

In [24]:
prix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28552954 entries, 0 to 28552953
Data columns (total 6 columns):
annee     category
id_pdv    category
nom       category
id        category
maj       datetime64[ns]
valeur    int64
dtypes: category(4), datetime64[ns](1), int64(1)
memory usage: 572.2 MB


In [25]:
prix

,annee,id_pdv,nom,id,maj,valeur
0,2017,1000001,Gazole,1,2017-01-02 09:37:03.000000,1258
1,2017,1000001,Gazole,1,2017-01-03 09:54:58.000000,1258
2,2017,1000001,Gazole,1,2017-01-06 12:33:57.000000,1258
3,2017,1000001,Gazole,1,2017-01-09 08:59:53.000000,1258
4,2017,1000001,Gazole,1,2017-01-10 10:38:39.000000,1258
...,...,...,...,...,...,...
28552949,2012,95870008,E10,5,2012-12-24 08:12:41.125140,1446
28552950,2012,95870008,E10,5,2012-12-26 09:59:50.402522,1453
28552951,2012,95870008,E10,5,2012-12-27 06:37:54.702091,1449
28552952,2012,95870008,E10,5,2012-12-28 10:30:37.417588,1448


- **enregistrer la df compréssée**

In [26]:
prix.to_parquet('prix.gzip',
               compression='gzip')

- **liberer la mémoire**

In [27]:
del prix

### netoyage de **services**

In [28]:
services = pd.read_csv('services.csv', low_memory=False)

In [29]:
services

,annee,id_pdv,service
0,2017,1000001,Station de gonflage
1,2017,1000001,Vente de gaz domestique
2,2017,1000001,Automate CB
3,2017,1000002,Vente de gaz domestique
4,2017,1000002,Automate CB
...,...,...,...
837086,2012,95870008,Piste poids lourds
837087,2012,96600001,Boutique non alimentaire
837088,2012,96600001,Vente de gaz domestique
837089,2012,96600001,rieure


In [30]:
services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837091 entries, 0 to 837090
Data columns (total 3 columns):
annee      837091 non-null int64
id_pdv     837091 non-null int64
service    837091 non-null object
dtypes: int64(2), object(1)
memory usage: 19.2+ MB


- **convertion des categories**

In [31]:
findLargeCols(services)

col_name                            n               n_distinct     ratio
annee                              837091                   13    64392%
id_pdv                             837091                15179       55%
service                            837091                  172     4867%


les columns **'annee','id_pdv','service'** mieux vaut les considérer des categories.

In [32]:
services["annee"]   = services["annee"].astype('category')
services["id_pdv"]  = services["id_pdv"].astype('category')
services["service"] = services["service"].astype('category')

In [33]:
services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837091 entries, 0 to 837090
Data columns (total 3 columns):
annee      837091 non-null category
id_pdv     837091 non-null category
service    837091 non-null category
dtypes: category(3)
memory usage: 4.7 MB


In [34]:
services

,annee,id_pdv,service
0,2017,1000001,Station de gonflage
1,2017,1000001,Vente de gaz domestique
2,2017,1000001,Automate CB
3,2017,1000002,Vente de gaz domestique
4,2017,1000002,Automate CB
...,...,...,...
837086,2012,95870008,Piste poids lourds
837087,2012,96600001,Boutique non alimentaire
837088,2012,96600001,Vente de gaz domestique
837089,2012,96600001,rieure


- **enregistrer la df compréssée**

In [35]:
services.to_parquet('services.gzip',
               compression='gzip')

- **liberer la mémoire**

In [36]:
del services